<a href="https://colab.research.google.com/github/luciadeangulo/UFV-Visualizacion/blob/main/clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EJERCICIO HISTÓRICO DE VENTAS: PLANIFICACIÓN

1 - Definición mensaje a transmitir
2 - Conocer los datos

In [2]:
import pandas as pd
df = pd.read_csv('datos_ejercicio_ventas.csv')
print(df.head())

         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT  
0         2023.0  754356.237194  
1         2023.0  560030.558029  
2         2023.0   88501.980847  
3         2023.0  363224.511516  
4         2023.0  396176.120491  


**Amount ** : volumen ventas en litros

**subbrand ** : producto que se ha vendido (lipton ej)

**scenario:** Ai_forecast(se ha predicho) o actual(venta real)

**month** : en caso de forecast, esto indica para cuándo se hace la predicción

**forecast** : mes cuando se hizo la predicción.  numero-1 = mes (2-12). pero diciembre es solo PF.

** forecast_year** : año cuando se hizo la predicción

PREGUNTAS PREVIAS PARA ENTENDER LOS DATOS:

1. Número actuals y forecasts

2. Horizonte de predicción (si hago previsión para enero y febrero, ej, el horizonte de predicción es 2, con frecuencia mensual): tiempo hacia el futuro para el cual se hacen previsiones.

3. Nº países y de productos

4. Histórico de actuals y forecasts

5. Forecasts distintos.

In [5]:
!pip install plotly_express
import plotly.express as px

1. Números Actuals y Forecasts

In [6]:
scenarios = df['SCENARIO'].value_counts()
scenarios

,count
SCENARIO,
AI_forecast,17766
actual,900


2. Horizonte de predicción

In [8]:
#solo tengo en cuenta los registros que son predicciones
predicciones = df[df['SCENARIO'] == 'AI_forecast']
#último año + mes de las predicciones totales
#uso variable year porque quiero el año para el cual predigo, no en el que predigo
max_year = predicciones['YEAR'].max()
max_month = predicciones[predicciones['YEAR'] == max_year]['MONTH'].max()
max_year, max_month

(2025, 5)

Ya tengo el mes (mayo) y el año (2025) para cuando se ha hecho la última predicción.  Ahora necesito el último mes+año de 'actual' para ver cuál es el último dato 'real' de ventas.

In [9]:
actuals = df[df['SCENARIO'] == 'actual']
max_actual_year = actuals['YEAR'].max()
max_actual_month = actuals[actuals['YEAR'] == max_actual_year]['MONTH'].max()
max_actual_year,max_actual_month

(2024, 8)

Ahora calculo la diferencia entre ambos:

In [11]:
horizonte_years = max_year - max_actual_year
horizonte_months = max_month - max_actual_month


(1, -3)

In [13]:
# Si los meses son negativos, ajustar los años
if horizonte_months < 0:
    horizonte_years -= 1
    horizonte_months += 12

In [14]:
horizonte_years,horizonte_months

(0, 9)

El horizonte de predicción es de 9 meses.

3.  Número de países y productos

In [18]:
paises = df['COUNTRY'].value_counts()
productos = df['SUBBRAND'].value_counts()
paises, productos

(COUNTRY
 Great Britain    4008
 Netherlands      2476
 Hungary          2384
 Denmark          2294
 Spain            2152
 Norway           2062
 Czech            1403
 Portugal         1179
 Italy             708
 Name: count, dtype: int64,
 SUBBRAND
 Pepsi Max (L3)        5598
 Pepsi Regular (L3)    3636
 7up (L3)              2694
 Lipton (L3)           2693
 7up Free (L3)         2343
 Mountain Dew (L3)     1702
 Name: count, dtype: int64)

Hay 9 países y 6 marcas de bebidas.

**PREGUNTAS A RESPONDER**
1. Cómo se distribuyen las ventas realizadas en cada país, mes y año y marca.

2. Cuál es la tendencia y estacionalidad de todas las ventas del país con menos ventas y la marca con más ventas.

3.  Cuáles son las predicciones hechas en España y qué tan buenas son.

******************************************************************

1. Cómo se distribuyen las ventas realizadas en cada país, mes y año y marca.

In [22]:
long_df = px.data.medals_long()
fig = px.bar(df, x="COUNTRY", y="AMOUNT", color="SUBBRAND", title="Ventas por País")
fig.show


<bound method BaseFigure.show of Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'SUBBRAND=Lipton (L3)<br>COUNTRY=%{x}<br>AMOUNT=%{y}<extra></extra>',
              'legendgroup': 'Lipton (L3)',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Lipton (L3)',
              'offsetgroup': 'Lipton (L3)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Portugal', 'Great Britain', 'Hungary', ..., 'Great Britain',
                          'Great Britain', 'Portugal'], dtype=object),
              'xaxis': 'x',
              'y': array([754356.23719405, 560030.55802897, 396176.12049052, ..., 912448.11274067,
                          700593.75932195, 533063.40364508]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'SUBBRAND=Pepsi Max (L3)<br>COUNTRY=%{x}<br>AMOUNT=%{y}<extra></extra>',
              'legendgroup': 'Pepsi Max (L3)',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Pepsi Max (L3)',
              'offsetgroup': 'Pepsi Max (L3)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Spain', 'Great Britain', 'Norway', ..., 'Hungary', 'Hungary',
                          'Denmark'], dtype=object),
              'xaxis': 'x',
              'y': array([   88501.98084743, 11385568.69333082,  2607242.89302767, ...,
                            497908.49331219,   479852.68697723,  2087623.78657171]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'SUBBRAND=7up (L3)<br>COUNTRY=%{x}<br>AMOUNT=%{y}<extra></extra>',
              'legendgroup': '7up (L3)',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': '7up (L3)',
              'offsetgroup': '7up (L3)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Great Britain', 'Denmark', 'Czech', ..., 'Netherlands', 'Netherlands',
                          'Norway'], dtype=object),
              'xaxis': 'x',
              'y': array([363224.51151579,      0.        ,  64054.56981421, ..., 123927.09312953,
                          150515.77609818,      0.        ]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'SUBBRAND=Pepsi Regular (L3)<br>COUNTRY=%{x}<br>AMOUNT=%{y}<extra></extra>',
              'legendgroup': 'Pepsi Regular (L3)',
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': 'Pepsi Regular (L3)',
              'offsetgroup': 'Pepsi Regular (L3)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Norway', 'Italy', 'Hungary', ..., 'Netherlands', 'Great Britain',
                          'Hungary'], dtype=object),
              'xaxis': 'x',
              'y': array([  26672.67485764, 2074742.36945837,  774392.1831584 , ...,
                           468362.73521127, 1313511.4398266 , 1314395.00849149]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'SUBBRAND=Mountain Dew (L3)<br>COUNTRY=%{x}<br>AMOUNT=%{y}<extra></extra>',
              'legendgroup': 'Mountain Dew (L3)',
              'marker': {'color': '#FFA15A', 'pattern': {'shape': ''}},
              'name': 'Mountain Dew (L3)',
              'offsetgroup': 'Mountain Dew (L3)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Netherlands', 'Netherlands'